In [1]:
import pandas as pd
import numpy as np
import time
import os
from datetime import datetime, timedelta, date, time
from datetime import timedelta
from datetime import datetime
import dateutil.parser as parser
from openpyxl import load_workbook

In [2]:
#Variables

usuario = os.path.expanduser('~')

fecha= datetime.now()
fecha_hoy = fecha.strftime("%m%d%y")

#fecha_hoy = '062623' Utilizar esta linea modificando el valor fecha "MDA" en caso de correr el archivo fuera de la fecha de ejecución.

In [3]:
#Ruta y archivo entregado por la empresa

ruta_archivo = f"{usuario}\\Downloads\\Activity_Master.xlsx"
ruta_descarga = f"{usuario}\\Desktop\\Proyectos\\ezlynx\\Entregables\\"

#Verificación de ruta

test1 = os.path.isdir(ruta_descarga) #Verifica que exista la carpeta

if test1 != True:
    os.mkdir(ruta_descarga) #Crea la carpeta inexistente

In [4]:
data=pd.read_excel(ruta_archivo, engine='openpyxl', dtype={'Policy Number': 'str'}).fillna('')

data['xm12'] = pd.to_datetime('today')

##Filtro para que el reporte sólo incluya los tasks que vencen hasta dentro de 6 días

data = data[data['Task - Due Date'] < data.xm12 + timedelta(6)]

data['Status'] = np.where((data['Task - Due Date'] < data.xm12), "Overdue", "Next")

data = data.drop_duplicates(subset=['Applicant ID', 'Policy Number'], keep='first')

data.info()
print(data)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 211 entries, 34 to 4974
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Applicant ID             211 non-null    int64         
 1   Account Name             211 non-null    object        
 2   Assigned Agent           211 non-null    object        
 3   Activity ID              211 non-null    int64         
 4   Activity Type            211 non-null    object        
 5   Date Created             211 non-null    datetime64[ns]
 6   Discussion               211 non-null    object        
 7   Note                     211 non-null    object        
 8   Policy Number            211 non-null    object        
 9   Task                     211 non-null    object        
 10  Task - Assigned To       211 non-null    object        
 11  Task - Due Date          211 non-null    datetime64[ns]
 12  Task - Last Modified     211 non-n

In [5]:
#Creación de tabla dinamica

t1 = pd.pivot_table(data, values='Activity ID', index='Labels', columns=['Task - Assigned To', 'Status'], aggfunc='count', fill_value=0)

t2 = pd.DataFrame(t1.to_records()).fillna('')

t2 = t2.rename(columns={"('Brenda assistant assistant of Victor Dueri', 'Next')": "Brenda_Next", "('Brenda assistant assistant of Victor Dueri', 'Overdue')":"Brenda_Overdue", 
                        "('Samuel assistant of Victor Dueri', 'Next')": "Samuel_Next",  "('Samuel assistant of Victor Dueri', 'Overdue')": "Samuel_Overdue",
                        "('Paola assistant  of Victor Dueri', 'Next')" : "Paola_Next",  "('Paola assistant  of Victor Dueri', 'Overdue')":  "Paola_Overdue",
                        "('Victor Dueri', 'Next')" : "Victor_Next",  "('Victor Dueri', 'Overdue')":  "Victor_Overdue"})

t2['Next'] = t2['Brenda_Next'] + t2['Paola_Next'] + t2['Samuel_Next'] + t2['Victor_Next']
t2['Overdue'] = t2['Brenda_Overdue'] + t2['Paola_Overdue'] + t2['Samuel_Overdue'] + t2['Victor_Overdue']

arrayt2 = np.array(t2.sum(numeric_only = True))
t3 = pd.DataFrame([arrayt2], columns = ["Brenda_Next", "Brenda_Overdue", "Paola_Next", "Paola_Overdue", "Samuel_Next", "Samuel_Overdue", "Victor_Next", "Victor_Overdue", "Next", "Overdue"
])
t4 = pd.concat([t2, t3])

In [6]:
writer = pd.ExcelWriter(f'{ruta_descarga}Open tasks VD {fecha_hoy}.xlsx', datetime_format='MM/DD/YYYY', date_format='MMDDYYYY')

t4.to_excel(writer, sheet_name='Resumen', startrow=4, startcol=1, index=False)

workbook = writer.book
worksheet = writer.sheets['Resumen']
header_names = [{'header': t4.columns[x]} for x in range(t4.shape[1])]
text = f'OPEN TASKS'
title_format = workbook.add_format()
title_format.set_font_size(20)
worksheet.write(1, 1, text, title_format)

worksheet.add_table('B5:L' + str(t4.shape[0] + 4 + 1), {'columns': header_names, 'style': 'Table Style Light 10'})

        
for column in t4:
    num_format = workbook.add_format({'num_format': '#,###'})
    worksheet.set_column(2, 13, None, num_format)

### I Detalle

data= data[['Applicant ID', 	'Account Name', 	'Date Created', 	'Discussion ', 	'Note', 'Activity ID',	'Policy Number', 	'Task', 
            'Task - Assigned To', 	'Task - Due Date', 	'Task - Last Modified', 	'Task - Status', 	'Labels', 	
             	'Task - Last Modified By', 	'Status', ]]

data.to_excel(writer, sheet_name='Open_tasks', startrow=4, startcol=0, index=False)
workbook = writer.book

worksheet = writer.sheets['Open_tasks']
header_names = [{'header': data.columns[x]} for x in range(data.shape[1])]
text = f'OPEN TASKS'
title_format = workbook.add_format()
title_format.set_font_size(20)
worksheet.write(0, 0, text, title_format)
        
worksheet.add_table('A5:P' + str(data.shape[0] + 4 + 1), {'columns': header_names, 'style': 'Table Style Light 10'})

for column in data:
    num_format = workbook.add_format({'num_format': '#,###'})
    # Apply the number format to the specified columns
    worksheet.set_column('G:G', None, num_format)
        #worksheet.set_column(0, 0, 10)    
    worksheet.freeze_panes(5, 3)  # Freeze the first row.


writer._save()

In [7]:
#Auto ajustar columnas

archivo = f'{ruta_descarga}Open tasks VD {fecha_hoy}.xlsx'
wb = load_workbook(archivo)
worksheet = wb['Open_tasks']

for i, col in enumerate(data.columns):
    column_width = max(data[col].astype(str).map(len).max(), len(col)) + 2
    worksheet.column_dimensions[worksheet.cell(row=1, column=i + 1).column_letter].width = column_width

wb.save(archivo)

In [8]:
print("Script Finalizado")

Script Finalizado
